In [14]:
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import imageio
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext


SEED = 1
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [1]:
# Prior to the tutorial, run this cell (to download the dataset we'll use)
TEXT = torchtext.data.Field(tokenizer_language = 'basic_english')
LABEL = torchtext.data.LabelField(dtype = torch.float)
train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL) # load IMDB dataset

TEXT.build_vocab(train_data, vectors='glove.6B.50d')
LABEL.build_vocab(train_data)

# Computing Word Embeddings

In [1]:
CONTEXT_SIZE = 2 # Number of context words used to predict the next word
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)

trigrams = 

# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = # Vocabulary
word_to_ix = # word to index dictionary

class NGramLanguageModeler(nn.Module):
    # GOAL: Predict the next word from the previous 2 words

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings =  # word embeddings
        self.linear1 =  # map 2 context embeddings to a hidden layer of dimension 128
        self.linear2 =  # map from 128-dim hidden layer to a probability distribution over all the vocabulary

    def forward(self, inputs):
        embeds =  # inputs are indices, so need to get corresponding embeddings first
        out =  # map to hidden layer
        out =  # get output
        log_probs =  # log probabilities
        return log_probs


losses = []
loss_function = # cross-entropy loss for word classification
model =  # initialize the model
optimizer = # use SGD as the optimizer - we are learning the embeddings and the linear1 & linear2 layers

# training loop
for epoch in range(50):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs =  # get 2 context words indices

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = 

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = 

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

In [ ]:
# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

In [20]:
# TSNE visualization of embeddings
embs = torch.stack([model.embeddings.weight[word_to_ix[word]] for word in vocab]).detach().numpy()
labels = [word for word in vocab] #

embs_2d = TSNE(n_components=2).fit_transform(embs)

fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(embs_2d[:,0], embs_2d[:,1])

for i, txt in enumerate(labels):
    ax.annotate(txt, (embs_2d[i,0], embs_2d[i,1]))

# Sentiment Classification on the IMDB dataset

In [22]:
TEXT = torchtext.data.Field(tokenizer_language = 'basic_english')
LABEL = torchtext.data.LabelField(dtype = torch.float)
train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL) # load IMDB dataset

In [2]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [25]:
print(vars(train_data.examples[0]))

{'text': ['We', 'do', 'not', 'come', 'across', 'movies', 'on', 'brother-sister', 'relationship', 'in', 'Indian', 'cinema,', 'or', 'any', 'other', 'language', 'or', 'medium.', 'This', 'relationship', 'has', 'several', 'aspects', 'which', 'have', 'not', 'been', 'exploited', 'in', 'movies', 'or', 'novels.', 'Typically,', 'a', 'sister', 'is', 'depicted', 'as', 'a', 'pile-on', 'who', 'can', 'be', 'used', 'for', 'ransom', 'in', 'the', 'climax.', 'This', 'movie', 'treats', 'the', 'subject', 'in', 'an', 'entirely', 'different', 'light.<br', '/><br', '/>It', 'is', 'inspired', 'by', 'George', "Eliot's", 'novel', '"The', 'Mill', 'on', 'the', 'Floss".', 'The', 'brother', 'is', 'very', 'prosaic,', 'all-good,', 'the', 'blue-eyed', 'boy', 'who', 'is', 'a', 'conventionally', 'good', 'son', 'and', 'a', 'favorite', 'with', 'his', 'mother.', 'The', 'sister', 'is', 'romantic,', 'wild', 'and', 'defiant', 'of', 'the', 'unwritten', 'rules', 'of', 'the', 'society.', 'In', 'spite', 'of', 'this,', 'the', 'love'

In [28]:
TEXT.build_vocab(train_data, vectors='glove.6B.50d')
LABEL.build_vocab(train_data)

In [29]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 224776
Unique tokens in LABEL vocabulary: 2


In [30]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 201300), ('a', 109208), ('and', 107232), ('of', 100305), ('to', 92934), ('is', 72631), ('in', 59955), ('I', 46058), ('that', 45393), ('this', 40088), ('it', 37880), ('/><br', 35840), ('was', 32602), ('as', 30054), ('with', 29360), ('for', 28760), ('but', 23528), ('The', 23463), ('on', 21516), ('movie', 21316)]


In [31]:
def get_batch_sentence_embeddings(batch):
    # write a function that, given a batch of text, computes a sentence embedding by averaging word embeddings
    sentence_embeddings = []
    for text in batch.text.transpose(1,0):
        filtered_text = text[(text!=0)&(text!=1)] # removes <unk> and <pad> tokens
        sentence_emb = # take the mean of the word embeddings for each sentence
        sentence_embeddings.append(sentence_emb)
    
    return # convert from list of tensors to tensor and return

class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = # linear layer

    def forward(self, x):
        outputs = # linear layer
        return outputs

In [32]:
# dataloader
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

# model + loss + optimizer
model = # create the model
criterion = torch.nn.BCEWithLogitsLoss() # cross-entropy loss for 0-1 sentiment classification
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2) # again, use SGD as the optimizer

In [ ]:
epochs = 30

losses = []
for epoch in range(int(epochs)):
    for i,batch in tqdm(enumerate(train_iterator)):
        
        sentence_embeddings = # compute sentence embeddings for the batch
        optimizer.zero_grad() # zero the gradients
        outputs =  # make a sentiment prediction
        loss =  # compute loss
        # compute gradients
        # update model parameters
        
    losses.append(loss.item())

print(losses)

In [36]:
def compute_metrics(iterator,model):
    # compute AUC
    model.eval()
    all_preds = []
    all_labels = []
    for batch in iterator:
        sentence_embeddings = get_batch_sentence_embeddings(batch)
        with torch.no_grad():
            all_preds.extend(list(model(sentence_embeddings).detach().numpy()))
        all_labels.extend(list(batch.label.detach().numpy()))
    
    return {'auc': roc_auc_score(all_labels,all_preds)}

In [37]:
compute_metrics(test_iterator,model)

{'auc': 0.7285463104000001}

## References

Guthrie, Robert. “Word Embeddings: Encoding Lexical Semantics¶.” Word Embeddings: Encoding Lexical Semantics - PyTorch Tutorials 1.8.1+cu102 Documentation, pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py.

Trevett, Ben. “Bentrevett/Pytorch-Sentiment-Analysis.” GitHub, github.com/bentrevett/pytorch-sentiment-analysis. 